In [ ]:
# 스팸 데이터 (https://raw.githubusercontent.com/ukairia777/tensorflow-nlp-tutorial/main/10.%20RNN%20Text%20Classification/dataset/spam.csv)에 대해서 다음을 답하세요.

# 1. 데이터/텍스트 전처리를 하세요.(ham/spam 인코딩, 결측치, 중복데이터, 숫자 및 특수문자 제거 등)[20]

# 2. TfidfVectorizer와 LogisticRegression을 이용하여 이진 분류를 하되, 최적의   파라메터를 도출하고 분류 정확도를 표시하세요.[30]

In [1]:
import pandas as pd
url = 'https://raw.githubusercontent.com/ukairia777/tensorflow-nlp-tutorial/main/10.%20RNN%20Text%20Classification/dataset/spam.csv'

In [ ]:
# 1. 데이터/텍스트 전처리를 하세요.(ham/spam 인코딩, 결측치, 중복데이터, 숫자 및 특수문자 제거 등)[20]

In [2]:
df = pd.read_csv(url, encoding='latin1')
df.head(3)

,v1,v2,Unnamed: 2,Unnamed: 3,Unnamed: 4
0,ham,"Go until jurong point, crazy.. Available only ...",NaN,NaN,NaN
1,ham,Ok lar... Joking wif u oni...,NaN,NaN,NaN
2,spam,Free entry in 2 a wkly comp to win FA Cup fina...,NaN,NaN,NaN


In [3]:
# Selection
df = df[['v1', 'v2']]
df.head(3)

,v1,v2
0,ham,"Go until jurong point, crazy.. Available only ..."
1,ham,Ok lar... Joking wif u oni...
2,spam,Free entry in 2 a wkly comp to win FA Cup fina...


In [4]:
# 결측치 확인
df.isna().sum()

v1    0
v2    0
dtype: int64

In [5]:
# 중복 데이터 확인
df.shape, df.v2.nunique()

((5572, 2), 5169)

In [6]:
# 중복 데이터 제거 
df.drop_duplicates(subset=['v2'], inplace=True)
df.shape

(5169, 2)

In [7]:
# 구둣점, 숫자 제거
df.v2 = df.v2.str.replace('[^A-Za-z]', ' ', regex=True)

In [8]:
df.v2[0]

'Go until jurong point  crazy   Available only in bugis n great world la e buffet    Cine there got amore wat   '

In [ ]:
# 2. TfidfVectorizer 와 LogisticRegression을 이용하여 이진 분류를 하되, 최적의   파라메터를 도출하고 분류 정확도를 표시하세요.[30]

In [9]:
# ['ham', 'spam'] --> [0, 1]
df.v1 = df.v1.replace(['ham', 'spam'], [0, 1])
df.head(3)

,v1,v2
0,0,Go until jurong point crazy Available only ...
1,0,Ok lar Joking wif u oni
2,1,Free entry in a wkly comp to win FA Cup fina...


In [10]:
from sklearn.model_selection import train_test_split 
X_train, X_test, y_train, y_test = train_test_split(
    df.v2.values, df.v1.values, stratify=df.v1.values, test_size=0.2, random_state=2023
)
X_train.shape, X_test.shape, y_train.shape, y_test.shape

((4135,), (1034,), (4135,), (1034,))

In [11]:
from sklearn.pipeline import Pipeline 
from sklearn.model_selection import GridSearchCV 
from sklearn.feature_extraction.text import TfidfVectorizer 
from sklearn.linear_model import LogisticRegression 

In [12]:

tvect = TfidfVectorizer(stop_words='english')
lrc = LogisticRegression(random_state=2023)

params = {
    'TVECT__max_df': [100,500],
    'LRC__C': [1,10]
}

pipeline = Pipeline([ ('TVECT', tvect),('LRC', lrc)])
grid_pipe = GridSearchCV(pipeline, params, scoring='accuracy', cv=3)
grid_pipe.fit(X_train, y_train)
grid_pipe.best_params_

{'LRC__C': 10, 'TVECT__max_df': 500}

In [13]:
grid_pipe.best_estimator_.score(X_test, y_test)

0.9709864603481625